In [1]:
# This notebook was created to do validation on collaboration filtering models
# It relies on a data extract in certain formats to operate, although there is a mock data generator
#
# There are 2 main functions:
#
# ** collab_recall_validation_tester()
# Input: customer purchase data, testing prarameters
# K-fold will be used for validation, using the (k-1)/k of the customer purcahse data to build a co-occurrence
# matrix and validating with 1/k of the data. The 1/k of the data will have X number of purchases removed
# for each customer and try to recommend the missing purchase using the co-occurrence matrix. The limitation
# is that any users with less than X purchases will not be used to calculate the accurracy.
# Output: Recall Accurracy
# **
#
# ** collab_catalog_validation_tester()
# Input: customer purchase data, testing prarameters
# K-fold will be used for validation, using the (k-1)/k of the customer purcahse data to build a co-occurrence
# matrix and validating with 1/k of the data. The 1/k of the data will keep all the original customer purchases
# and use it to recommend products using the co-occurrence matrix. Taking all the recommended products and 
# dividing it by the total amount of products offered will give us catalog coverage.
# Output: Catalog Coverage
# **
#
# Testing with categories is not implemented yet

In [2]:
import numpy as np
import random
import time
from sklearn.model_selection import KFold
import json
import sys
from datetime import date, datetime
import pandas as pd

In [3]:
# Create co-occurrence matrix

def create_cocmatrix(subset_matrix_cust_order):
    rows, cols = subset_matrix_cust_order.shape
    m = np.zeros((cols,cols))
    
    for i in range(cols):
        t = np.sum(subset_matrix_cust_order[subset_matrix_cust_order[:,i] > 0],axis=0)
        t[i] = 0
        m[i,:] = t
        
        #if i % (cols/5) == 0:
        #    print "created rows in cooccurrence matrix at row",i
            
    return m

In [4]:
def create_seed(rand):
    np.random.seed(rand) # create seed for repeatable results

In [5]:
# initialize all data for collaborative
# customer to demographic lookup
arr_cxd = np.load('demo_matrix.npy')

print "created customer to demographic lookup"

# customer to order lookup
matrix_co = np.load('cust_item_matrix.npy')

print "created customer to order lookup"

regions = []

with open("regions.txt", 'r') as regioninputs:
    for r in regioninputs:
        r = r.strip()
        try:
            rname, rid = r.split('\t')
        except:
            continue
            
        regions.append(float(rid))
        
regions = list(set(regions))

created customer to demographic lookup
created customer to order lookup


In [6]:
# print statistics
print "\ncustomer x demographic shape:",arr_cxd.shape
print "customer regions + unknown:",len(np.unique(arr_cxd[:,0]))

for num, name in enumerate(np.unique(arr_cxd[:,0])):
    print "\tregion",name,"% of pop",len(np.where(arr_cxd[:,0] == name)[0])/float(arr_cxd.shape[0])*100
    
print "customer genders + unknown:",len(np.unique(arr_cxd[:,1]))

for num, name in enumerate(np.unique(arr_cxd[:,1])):
    print "\tgender",name,"% of pop",len(np.where(arr_cxd[:,1] == name)[0])/float(arr_cxd.shape[0])*100
    
print "\ncustomer x purchases shape:",matrix_co.shape # customer and purchases
print "if already normalized the stats should be the same"
print "not normalized"

stat_sum_not_normal = np.sum(matrix_co,axis=1)

print "customer average purchases (multiples of item allowed):",np.mean(stat_sum_not_normal)
print "customer variance purchases (multiples of item allowed):",np.var(stat_sum_not_normal)
print "customer min purchases (multiples of item allowed):",np.min(stat_sum_not_normal)
print "customer max purchases (multiples of item allowed):",np.max(stat_sum_not_normal)
print "normalized"

stat_sum_normal = np.count_nonzero(matrix_co,axis=1)

print "customer average purchases (multiples of item set to 1):",np.mean(stat_sum_normal)
print "customer variance purchases (multiples of item set to 1):",np.var(stat_sum_normal)
print "customer min purchases (multiples of item set to 1):",np.min(stat_sum_normal)
print "customer max purchases (multiples of item set to 1):",np.max(stat_sum_normal)
print "customers that bought at least 1 item:",len(np.where(np.sum(matrix_co,axis=1) > 0)[0])
print "customers that bought at least 2 item:",len(np.where(np.sum(matrix_co,axis=1) > 1)[0])
print "customers that bought at least 3 item:",len(np.where(np.sum(matrix_co,axis=1) > 2)[0])
print "customers that bought at least 4 item:",len(np.where(np.sum(matrix_co,axis=1) > 3)[0])
print "customers that bought at least 5 item:",len(np.where(np.sum(matrix_co,axis=1) > 4)[0])


customer x demographic shape: (189559, 2)
customer regions + unknown: 22
	region 0.0 % of pop 0.193079727156
	region 1.0 % of pop 10.3403162076
	region 2.0 % of pop 42.9655146946
	region 3.0 % of pop 8.69280804393
	region 4.0 % of pop 2.49262762517
	region 5.0 % of pop 14.3607003624
	region 6.0 % of pop 1.45653859748
	region 7.0 % of pop 4.01194351099
	region 8.0 % of pop 3.41318534071
	region 9.0 % of pop 11.9308500256
	region 10.0 % of pop 0.0949572428637
	region 11.0 % of pop 0.00474786214318
	region 12.0 % of pop 0.0290147130972
	region 13.0 % of pop 0.00896818404824
	region 15.0 % of pop 0.000527540238132
	region 18.0 % of pop 0.00105508047626
	region 20.0 % of pop 0.000527540238132
	region 21.0 % of pop 0.000527540238132
	region 22.0 % of pop 0.000527540238132
	region 28.0 % of pop 0.000527540238132
	region 30.0 % of pop 0.000527540238132
	region 35.0 % of pop 0.000527540238132
customer genders + unknown: 3
	gender 0.0 % of pop 8.54826201869
	gender 1.0 % of pop 50.8976097152
	g

In [7]:
# generate recommendations
# purchase_vec = customer purchases (vector)
# cocm = co occurrence matrix [items x items]
# num_rec = number of recommendations
def gen_recom(purchase_list,cocm,num_rec):
    rowsum = np.zeros(cocm.shape[0])
    
    for p in purchase_list:
        rowsum += cocm[p,:]
        
    rowsum[purchase_list] = 0
    indices = np.nonzero(rowsum)[0]
    toprec = indices[np.argsort(rowsum[indices])][-1 * num_rec:][::-1]
    return toprec

In [8]:
def get_indices(dataset, col, var=None):
    if var is None:
        return [y for y, x in enumerate(dataset[:,col])] # all
    
    return [y for y, x in enumerate(dataset[:,col]) if x == var]

In [9]:
def check_idx_size(idxlist,maximum_rows):
    if len(idxlist) > maximum_rows:
        return np.random.choice(idxlist, maximum_rows, replace=False)
    
    return idxlist

In [10]:
#80/20, 80 for 10-fold cross validation, 20 for test set holdout
create_seed(0)
indices = np.random.permutation(matrix_co.shape[0])
tsize = int(matrix_co.shape[0]*.8)

# customer to order matrix
matrix_co_training_idx, matrix_co_testing_idx  = indices[:tsize], indices[tsize:]
matrix_co_training, matrix_co_testing = matrix_co[matrix_co_training_idx,:], matrix_co[matrix_co_testing_idx,:]

# demo matrix
arr_cxd_training_idx, arr_cxd_testing_idx = indices[:tsize], indices[tsize:]
arr_cxd_training, arr_cxd_testing = arr_cxd[arr_cxd_training_idx,:], arr_cxd[arr_cxd_testing_idx,:]

In [11]:
matrix_co_training.shape, arr_cxd_training.shape, matrix_co.shape, arr_cxd.shape

((151647, 3990), (151647, 2), (189559, 3990), (189559, 2))

In [12]:
# put matrixes in to get cross validation recall
# generate recommendations
# mco = matrix of customer to order [customer x items]
# num_rec = number of recommendations
# num_folds = k number of folds for cross validation
# recall_remove = removed number from purchase history
def collab_recall_validation_tester(mco,num_rec,num_folds,recall_remove):
    # cross validation n-folds
    start_time = time.time()
    kf = KFold(n_splits=num_folds)
    list_total_acc=[]
    k_index = 0
    
    for train, test in kf.split(mco):
        print "------------------------------------start new k=",k_index,\
        ",train set=",mco[train].shape[0],\
        ",validation set=",mco[test].shape[0]
        list_sub_acc = []
        # build co-occurrence matrix
        coo_matrix = create_cocmatrix(mco[train])

        # loop through test set
        # for each customer in test pool
        for i in range(mco[test].shape[0]):
            
            #print "customer",i
            # purchase vector
            #print "purchase vector"
            p_vec = mco[test][i,:]
            #print p_vec
            # determine purchase indexes
            #print "purchase indexes"
            list_original_purchases = np.where(p_vec > 0)[0]
            #print list_original_purchases
            
            # only run tests on customers with > recall_remove purchases for prediction
            if len(list_original_purchases) > recall_remove:
                # randomly select indexes to leave out
                #print "remove purchase indexes"
                #list_removed_purchases = np.random.choice(list_original_purchases,size=recall_remove, replace=False)
                list_removed_purchases = random.sample(list_original_purchases,recall_remove)
                #print list_removed_purchases
                # remove
                list_modified_purchases = list(set(list_original_purchases) - set(list_removed_purchases))
                #print "modified purchase vector"
                #p_vec_mod = np.zeros(p_vec.shape[0])
                #p_vec_mod[list_modified_purchases] = 1
                #print p_vec_mod
                # get sum all purchases except ones left out
                list_summed_coo_vec = gen_recom(list_modified_purchases,coo_matrix,num_rec)
                #print list_summed_coo_vec
                # check if return recommendations are in list
                list_recommended_match = set(list_removed_purchases) & set(list_summed_coo_vec)
                acc = len(list_recommended_match)/float(len(list_removed_purchases))
                #print "predicted ratio",accuracy
                list_sub_acc.append(acc)
                #print "end customer",i

                #if i % 500 == 0:
                # sanity check to make sure not all recommendations are #1
                #if (list_removed_purchases[0] != list_summed_coo_vec[0]) & (acc == 1):
                #    print "\t--customer example index",i
                #    print "\toriginal purchased indexes",list_original_purchases
                #    print "\tremoved index",list_removed_purchases
                #    print "\toriginal modified purchase indexes",np.where(p_vec_mod > 0)[0]
                #    print "\tpredicted indexes",list_summed_coo_vec     
                #    print "\tpredicted accuracy ratio",acc
                #    print "\t--customer end example index",i
            #else:
                #print "skipped due to length of purchases",list_original_purchases,\
                #"less than recall remove",recall_remove
        mean_sub_acc = np.mean(list_sub_acc)
        print "** number of elements in calculation",len(list_sub_acc)
        #print list_sub_acc
        print "** average fold accuracy",mean_sub_acc
        list_total_acc.append(mean_sub_acc)
        k_index += 1
        print "**time elapsed",(time.time() - start_time)
        print "------------------------------------end"
    print "list of total accuracy for each fold",list_total_acc
    average = np.mean(list_total_acc)
    print num_folds,"-fold total average",average
    print "time elapsed",(time.time() - start_time)
    return average

def demo_filter(mco,mdo,regions,maximum_rows,minimum_rows,num_rec,num_folds,recall_remove):
    list_info = []
    
    for r in regions:
        if r == 0.0:
            ridx = get_indices(mdo, 0) # use all regions
        else:   
            ridx = get_indices(mdo, 0, r) # look for region r in col 0

            if len(ridx) == 0:
                print "region: {0} not found".format(r)
                continue

        for g in [1.0,2.0,0.0]: #gender_list:
            if g == 0.0:
                gidx = get_indices(mdo, 1) # all, col 1 is gender
            else:
                gidx = get_indices(mdo, 1, g)

                if len(gidx) == 0:
                    print "gender: {0} not found".format(g)
                    continue

            interidx = list(set(ridx).intersection(set(gidx)))

            if len(interidx) < minimum_rows:
                print "too little data with region {0} and gender {1} found".format(r, g)
                continue

            interidx = check_idx_size(interidx,maximum_rows) # limit size
            print "\nregion: {0}, gender: {1}, intersect: {2}, {3}".format(r, g, len(interidx), len(mco))
                             
            total_accuracy = collab_recall_validation_tester(mco[interidx],num_rec,num_folds,recall_remove)
            list_info.append((r,g,mco[interidx].shape[0],total_accuracy))
    
    return list_info

In [13]:
# General test
seed = 0
create_seed(seed)
maximum_rows = 25000
minimum_rows = 1000

# test set
#randidx = np.random.choice(matrix_co_training.shape[0], 10000, replace=False)
#demo_results = demo_filter(matrix_co_training[randidx],arr_cxd_training[randidx],\
#                           regions,maximum_rows,minimum_rows,10,10,1)

# big set
demo_results = demo_filter(matrix_co_training,arr_cxd_training,\
                           regions,maximum_rows,minimum_rows,10,10,1)


region: 0.0, gender: 1.0, intersect: 25000, 151647
------------------------------------start new k= 0 ,train set= 22500 ,validation set= 2500
** number of elements in calculation 657
** average fold accuracy 0.630136986301
**time elapsed 99.2167019844
------------------------------------end
------------------------------------start new k= 1 ,train set= 22500 ,validation set= 2500
** number of elements in calculation 689
** average fold accuracy 0.641509433962
**time elapsed 188.74141407
------------------------------------end
------------------------------------start new k= 2 ,train set= 22500 ,validation set= 2500
** number of elements in calculation 683
** average fold accuracy 0.647144948755
**time elapsed 279.318017006
------------------------------------end
------------------------------------start new k= 3 ,train set= 22500 ,validation set= 2500
** number of elements in calculation 695
** average fold accuracy 0.631654676259
**time elapsed 368.374351025
-------------------------


region: 1.0, gender: 1.0, intersect: 7924, 151647
------------------------------------start new k= 0 ,train set= 7131 ,validation set= 793
** number of elements in calculation 206
** average fold accuracy 0.577669902913
**time elapsed 9.98088812828
------------------------------------end
------------------------------------start new k= 1 ,train set= 7131 ,validation set= 793
** number of elements in calculation 233
** average fold accuracy 0.570815450644
**time elapsed 20.060762167
------------------------------------end
------------------------------------start new k= 2 ,train set= 7131 ,validation set= 793
** number of elements in calculation 224
** average fold accuracy 0.59375
**time elapsed 29.8755362034
------------------------------------end
------------------------------------start new k= 3 ,train set= 7131 ,validation set= 793
** number of elements in calculation 236
** average fold accuracy 0.601694915254
**time elapsed 39.4674110413
------------------------------------end
-


region: 2.0, gender: 1.0, intersect: 25000, 151647
------------------------------------start new k= 0 ,train set= 22500 ,validation set= 2500
** number of elements in calculation 650
** average fold accuracy 0.667692307692
**time elapsed 89.9008989334
------------------------------------end
------------------------------------start new k= 1 ,train set= 22500 ,validation set= 2500
** number of elements in calculation 691
** average fold accuracy 0.671490593343
**time elapsed 179.173871994
------------------------------------end
------------------------------------start new k= 2 ,train set= 22500 ,validation set= 2500
** number of elements in calculation 698
** average fold accuracy 0.63323782235
**time elapsed 274.113818884
------------------------------------end
------------------------------------start new k= 3 ,train set= 22500 ,validation set= 2500
** number of elements in calculation 648
** average fold accuracy 0.645061728395
**time elapsed 367.919239998
-------------------------


region: 3.0, gender: 1.0, intersect: 6837, 151647
------------------------------------start new k= 0 ,train set= 6153 ,validation set= 684
** number of elements in calculation 199
** average fold accuracy 0.572864321608
**time elapsed 7.12711715698
------------------------------------end
------------------------------------start new k= 1 ,train set= 6153 ,validation set= 684
** number of elements in calculation 192
** average fold accuracy 0.526041666667
**time elapsed 14.2127420902
------------------------------------end
------------------------------------start new k= 2 ,train set= 6153 ,validation set= 684
** number of elements in calculation 189
** average fold accuracy 0.481481481481
**time elapsed 21.2192280293
------------------------------------end
------------------------------------start new k= 3 ,train set= 6153 ,validation set= 684
** number of elements in calculation 180
** average fold accuracy 0.6
**time elapsed 28.335078001
------------------------------------end
-----


region: 4.0, gender: 1.0, intersect: 1952, 151647
------------------------------------start new k= 0 ,train set= 1756 ,validation set= 196
** number of elements in calculation 57
** average fold accuracy 0.40350877193
**time elapsed 0.928881883621
------------------------------------end
------------------------------------start new k= 1 ,train set= 1756 ,validation set= 196
** number of elements in calculation 61
** average fold accuracy 0.393442622951
**time elapsed 1.86623597145
------------------------------------end
------------------------------------start new k= 2 ,train set= 1757 ,validation set= 195
** number of elements in calculation 67
** average fold accuracy 0.373134328358
**time elapsed 2.81937503815
------------------------------------end
------------------------------------start new k= 3 ,train set= 1757 ,validation set= 195
** number of elements in calculation 55
** average fold accuracy 0.509090909091
**time elapsed 3.73983693123
------------------------------------e

------------------------------------start new k= 0 ,train set= 10316 ,validation set= 1147
** number of elements in calculation 310
** average fold accuracy 0.596774193548
**time elapsed 19.439027071
------------------------------------end
------------------------------------start new k= 1 ,train set= 10316 ,validation set= 1147
** number of elements in calculation 355
** average fold accuracy 0.549295774648
**time elapsed 38.7424190044
------------------------------------end
------------------------------------start new k= 2 ,train set= 10316 ,validation set= 1147
** number of elements in calculation 350
** average fold accuracy 0.534285714286
**time elapsed 57.8108530045
------------------------------------end
------------------------------------start new k= 3 ,train set= 10317 ,validation set= 1146
** number of elements in calculation 310
** average fold accuracy 0.61935483871
**time elapsed 76.8960249424
------------------------------------end
------------------------------------st


region: 6.0, gender: 1.0, intersect: 1184, 151647
------------------------------------start new k= 0 ,train set= 1065 ,validation set= 119
** number of elements in calculation 25
** average fold accuracy 0.36
**time elapsed 0.367114782333
------------------------------------end
------------------------------------start new k= 1 ,train set= 1065 ,validation set= 119
** number of elements in calculation 30
** average fold accuracy 0.366666666667
**time elapsed 0.739233016968
------------------------------------end
------------------------------------start new k= 2 ,train set= 1065 ,validation set= 119
** number of elements in calculation 36
** average fold accuracy 0.527777777778
**time elapsed 1.10232591629
------------------------------------end
------------------------------------start new k= 3 ,train set= 1065 ,validation set= 119
** number of elements in calculation 38
** average fold accuracy 0.289473684211
**time elapsed 1.46287488937
------------------------------------end
-----

------------------------------------start new k= 0 ,train set= 2053 ,validation set= 229
** number of elements in calculation 58
** average fold accuracy 0.5
**time elapsed 1.14786696434
------------------------------------end
------------------------------------start new k= 1 ,train set= 2053 ,validation set= 229
** number of elements in calculation 67
** average fold accuracy 0.34328358209
**time elapsed 2.3108420372
------------------------------------end
------------------------------------start new k= 2 ,train set= 2054 ,validation set= 228
** number of elements in calculation 71
** average fold accuracy 0.380281690141
**time elapsed 3.46609592438
------------------------------------end
------------------------------------start new k= 3 ,train set= 2054 ,validation set= 228
** number of elements in calculation 72
** average fold accuracy 0.458333333333
**time elapsed 4.6268260479
------------------------------------end
------------------------------------start new k= 4 ,train set=

** number of elements in calculation 57
** average fold accuracy 0.456140350877
**time elapsed 0.961344003677
------------------------------------end
------------------------------------start new k= 1 ,train set= 1783 ,validation set= 199
** number of elements in calculation 49
** average fold accuracy 0.387755102041
**time elapsed 1.92147397995
------------------------------------end
------------------------------------start new k= 2 ,train set= 1784 ,validation set= 198
** number of elements in calculation 57
** average fold accuracy 0.491228070175
**time elapsed 2.88852190971
------------------------------------end
------------------------------------start new k= 3 ,train set= 1784 ,validation set= 198
** number of elements in calculation 58
** average fold accuracy 0.327586206897
**time elapsed 3.83019900322
------------------------------------end
------------------------------------start new k= 4 ,train set= 1784 ,validation set= 198
** number of elements in calculation 57
** aver

------------------------------------start new k= 0 ,train set= 6399 ,validation set= 712
** number of elements in calculation 194
** average fold accuracy 0.494845360825
**time elapsed 8.22655892372
------------------------------------end
------------------------------------start new k= 1 ,train set= 6400 ,validation set= 711
** number of elements in calculation 175
** average fold accuracy 0.491428571429
**time elapsed 16.3487889767
------------------------------------end
------------------------------------start new k= 2 ,train set= 6400 ,validation set= 711
** number of elements in calculation 179
** average fold accuracy 0.530726256983
**time elapsed 24.4665539265
------------------------------------end
------------------------------------start new k= 3 ,train set= 6400 ,validation set= 711
** number of elements in calculation 192
** average fold accuracy 0.505208333333
**time elapsed 32.6193618774
------------------------------------end
------------------------------------start ne

In [14]:
demo_results

[(0.0, 1.0, 25000, 0.63021129129005082),
 (0.0, 2.0, 25000, 0.58928032527908458),
 (0.0, 0.0, 25000, 0.61261214188248769),
 (1.0, 1.0, 7924, 0.58622166842109924),
 (1.0, 2.0, 6579, 0.53207443609961458),
 (1.0, 0.0, 15758, 0.5876985518511354),
 (2.0, 1.0, 25000, 0.65063788769009956),
 (2.0, 2.0, 25000, 0.62005193234070033),
 (2.0, 0.0, 25000, 0.64711728136894542),
 (3.0, 1.0, 6837, 0.53764313980411571),
 (3.0, 2.0, 5121, 0.51474304854702357),
 (3.0, 0.0, 13182, 0.56771567848345794),
 (4.0, 1.0, 1952, 0.41744365812589523),
 (4.0, 2.0, 1487, 0.42683877366095169),
 (4.0, 0.0, 3779, 0.48917797727751244),
 (5.0, 1.0, 11463, 0.56400391026070895),
 (5.0, 2.0, 8390, 0.5273902715671952),
 (5.0, 0.0, 21856, 0.58439760127801332),
 (6.0, 1.0, 1184, 0.38824033901404675),
 (6.0, 0.0, 2203, 0.4397824719079434),
 (7.0, 1.0, 3200, 0.48116420435288776),
 (7.0, 2.0, 2282, 0.42731026098927349),
 (7.0, 0.0, 6115, 0.5236092406630658),
 (8.0, 1.0, 2796, 0.4887248725036839),
 (8.0, 2.0, 1982, 0.418831732959984

In [15]:
np.save('demo_results_big_1k-25k.npy',[(maximum_rows,minimum_rows,seed,datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))]+demo_results)

In [16]:
demo_results_copy = np.load('demo_results_big_1k-25k.npy')
demo_results_copy

array([['25000', '1000', '0', '2017-11-30_03:01:44'],
       ['0.0', '1.0', '25000', '0.63021129129'],
       ['0.0', '2.0', '25000', '0.589280325279'],
       ['0.0', '0.0', '25000', '0.612612141882'],
       ['1.0', '1.0', '7924', '0.586221668421'],
       ['1.0', '2.0', '6579', '0.5320744361'],
       ['1.0', '0.0', '15758', '0.587698551851'],
       ['2.0', '1.0', '25000', '0.65063788769'],
       ['2.0', '2.0', '25000', '0.620051932341'],
       ['2.0', '0.0', '25000', '0.647117281369'],
       ['3.0', '1.0', '6837', '0.537643139804'],
       ['3.0', '2.0', '5121', '0.514743048547'],
       ['3.0', '0.0', '13182', '0.567715678483'],
       ['4.0', '1.0', '1952', '0.417443658126'],
       ['4.0', '2.0', '1487', '0.426838773661'],
       ['4.0', '0.0', '3779', '0.489177977278'],
       ['5.0', '1.0', '11463', '0.564003910261'],
       ['5.0', '2.0', '8390', '0.527390271567'],
       ['5.0', '0.0', '21856', '0.584397601278'],
       ['6.0', '1.0', '1184', '0.388240339014'],
       ['

In [17]:
drs = pd.DataFrame(demo_results_copy[1:],columns=['region','gender','size','accuracy'])

In [18]:
drs

,region,gender,size,accuracy
0,0.0,1.0,25000,0.63021129129
1,0.0,2.0,25000,0.589280325279
2,0.0,0.0,25000,0.612612141882
3,1.0,1.0,7924,0.586221668421
4,1.0,2.0,6579,0.5320744361
5,1.0,0.0,15758,0.587698551851
6,2.0,1.0,25000,0.65063788769
7,2.0,2.0,25000,0.620051932341
8,2.0,0.0,25000,0.647117281369
9,3.0,1.0,6837,0.537643139804
